In [94]:
import pandas as pd
import os
from skimage.transform import resize
from sklearn.utils.validation import column_or_1d
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
import h5py
import re
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor

scorer = make_scorer(mean_squared_error, greater_is_better=False)
param_grid={'C':[0.001],'gamma':[0.1],'kernel':['rbf']}
svr=svm.SVR()
model = svr
#model = SGDRegressor()
#model=GridSearchCV(svr, param_grid, scoring = scorer)

flat_data_arr=[] #input array
target_arr=[] #output array

#file path
path = 'C:/Users/a0981/OneDrive/桌面/archive/BraTS2020_training_data/content/data'
surInfoFile = "C:/Users/a0981/OneDrive/桌面/archive/BraTS2020_training_data/content/data/survival_info.csv"
MetaDataFile = "C:/Users/a0981/OneDrive/桌面/archive/BraTS2020_training_data/content/data/meta_data.csv"

date_column = ['Brats20ID','Age','Survival_days','Extent_of_Resection']
df_surInfo = pd.read_csv(surInfoFile,names= date_column,skiprows=1)
df_metaData = pd.read_csv(MetaDataFile,names= ['slice_path','target','volume','slice'],skiprows=1)


def modify_value(value):
    idStr = str(value)
    number_str = idStr[-3:] 
    number_int = int(number_str)
    return number_int

#for index,row in df_surInfo.iterrows():
 #   idStr = str(row['Brats20ID'])
  #  number_str = idStr[-3:] 
  #  number_int = int(number_str)
   # surInfoId.append(number_int)
    
df_surInfo['Brats20ID'] = df_surInfo['Brats20ID'].apply(modify_value)
print(df_surInfo)
print(df_metaData)






     Brats20ID     Age  Survival_days Extent_of_Resection
0            1  60.463            289                 GTR
1            2  52.263            616                 GTR
2            3  54.301            464                 GTR
3            4  39.068            788                 GTR
4            5  68.493            465                 GTR
..         ...     ...            ...                 ...
230        363  57.000             62                 GTR
231        366  72.000            633                 GTR
232        367  60.000            437                 STR
233        368  49.000            442                 GTR
234        369  50.000            145                 GTR

[235 rows x 4 columns]
                                  slice_path  target  volume  slice
0         /content/data/volume_41_slice_0.h5       0      41      0
1         /content/data/volume_41_slice_1.h5       0      41      1
2         /content/data/volume_41_slice_2.h5       0      41      2
3       

In [95]:
pattern = r"volume_(\d+)_"
pattern2 = r"slice_(\d+)"

y = pd.DataFrame(columns = ['Survival_days','Volume'])
volumes = []
#path which contains all the categories of images
fileCount = 0
volumeCount = 0
volumeCountMax = 20

volumeId = -1
for img in os.listdir(path):
    
    if img.endswith(".h5"):
        
        fileName = os.path.join(path,img)
        dataId = re.search(pattern, fileName).group(1)
        dataId = int(dataId)
        SliceId = re.search(pattern2, fileName).group(1)
        SliceId = int(SliceId)
        
        if dataId in df_surInfo['Brats20ID'].values:
            
            
            if volumeId != dataId :
                volumeId = dataId
                df_volume = df_metaData.loc[df_metaData['volume'] == volumeId, ['slice','target']]
                
                volumeCount += 1
                #print(df_volume)
            
            target = df_volume.loc[df_volume['slice'] == SliceId ,'target'].iloc[0]
            
            if volumeCount == volumeCountMax + 1:
            
                flat_data=np.array(flat_data_arr)
                df = pd.DataFrame(flat_data)

                #print(df)
                print(volumes)
                x=df.iloc[:,:-2] #input data 
                Y = df.iloc[:,-2]

                #print(x)
                #print(Y.ravel())
                
                num = int(volumeCountMax*0.8)
                train_volume = volumes[:num]
                test_volume = volumes[num:]
                print(train_volume)
                print(test_volume)
                
                df_train = df[df[df.columns[-1]].isin(train_volume)]
                #print(df_train)
                df_test = df[df[df.columns[-1]].isin(test_volume)]
                #print(df_test)
                
                x_train = df_train.iloc[:,:-2]
                y_train = df_train.iloc[:,-2]
                x_test = df_test.iloc[:,:-2]
                y_test = df_test.iloc[:,-2]
                        
                #x_train,x_test,y_train,y_test=train_test_split(x,Y,test_size=0.20,random_state=77)
                print('Splitted Successfully')
                print(x_train.shape)
                print(y_train.shape)
                print(x_test.shape)
                print(y_test.shape)
                #model.partial_fit(x_train,y_train)
                model.fit(x_train,y_train)
                print('The Model is trained well with the given images')

                y_pred=model.predict(x_test)
                print("The predicted Data is :")
                print(y_pred)
                print("The actual data is:")
                print(np.array(y_test))
                print(f"The model MSE is {mean_squared_error(y_pred,y_test)}")

                flat_data_arr = []
                y = pd.DataFrame(columns = ['Survival_days'])
                fileCount = 0
                volumeCount = 1
                volumes = []
            
            if target == 1:
                
                #print(f'dataId :{dataId}')
                #print(f'SliceId :{SliceId}')
                f = h5py.File(fileName, 'r')
                img_array = f.get('image')
                img_array = np.array(img_array)
                img_array = img_array.flatten()
                
                if dataId not in volumes:
                    volumes.append(dataId)
            
                age = df_surInfo[df_surInfo['Brats20ID']==dataId]['Age'].values[0]
                img_array = np.append(img_array,age)
                survDays = df_surInfo.loc[df_surInfo['Brats20ID'] == dataId, 'Survival_days'].iloc[0]
                img_array = np.append(img_array,survDays)
                img_array = np.append(img_array,dataId)
            
                flat_data_arr.append(img_array)
                fileCount += 1
            
        
        


[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 10, 110, 111, 112, 113, 114, 115, 116, 117, 118]
[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 10, 110, 111, 112, 113, 114]
[115, 116, 117, 118]
Splitted Successfully
(1019, 230401)
(1019,)
(259, 230401)
(259,)
The Model is trained well with the given images
The predicted Data is :
[416.13191167 416.14219142 415.95522766 415.86513425 415.76618715
 415.61497794 415.36526054 415.04517932 414.8195592  414.51803362
 414.105865   413.78052932 413.50737018 413.11214276 412.86892228
 412.559788   412.26399105 412.04534531 411.75514722 411.44234105
 411.029139   410.56934496 410.10331747 409.51629906 408.99519274
 408.35598546 407.59179022 406.84815292 406.14166653 405.59432708
 404.86188276 404.04231764 403.33343114 402.35774274 401.60478481
 400.92685093 405.42506071 405.7839061  405.86910797 405.76477769
 405.57295323 405.41759537 405.45857991 405.39117465 405.25706168
 404.93633758 404.61780045 404.49139272 404.33951162 404.36398378
 

[154, 157, 158, 159, 15, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 16, 170, 173, 174, 175]
[154, 157, 158, 159, 15, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 16]
[170, 173, 174, 175]
Splitted Successfully
(1117, 230401)
(1117,)
(279, 230401)
(279,)
The Model is trained well with the given images
The predicted Data is :
[358.37355302 358.49199211 358.49844207 358.60444088 358.48243018
 358.43983016 358.20031313 357.7233317  357.20665268 356.72066807
 356.32503606 355.92826122 355.56452198 355.24128048 355.01600553
 354.7676473  354.56274255 354.39986997 354.33947869 354.4081158
 354.58863892 354.93692697 355.60035682 356.10303417 356.6540997
 357.21127332 357.77678433 358.3291525  358.90128476 359.61142104
 360.61482694 361.57590013 362.57799875 363.42978188 364.32134782
 365.26685042 366.2620794  366.98975481 367.68375399 368.28267185
 368.91220455 369.37580546 369.75501774 370.08659237 370.31312013
 370.5371119  370.62800579 370.74085277 370.95864922 371.17687603
 371.

[196, 199, 19, 1, 201, 203, 204, 206, 20, 210, 214, 215, 216, 217, 218, 219, 21, 220, 221, 222]
[196, 199, 19, 1, 201, 203, 204, 206, 20, 210, 214, 215, 216, 217, 218, 219]
[21, 220, 221, 222]
Splitted Successfully
(1013, 230401)
(1013,)
(253, 230401)
(253,)
The Model is trained well with the given images
The predicted Data is :
[322.8234845  322.59603619 322.34675879 321.94889484 321.5015422
 321.27433826 321.04763311 320.93618321 320.80165048 320.6443734
 320.34411377 320.15261892 320.16037417 320.09205247 320.07519506
 320.11939728 320.03981071 319.89185267 319.83533844 319.80638506
 319.66297689 319.39738125 319.13922842 318.9003969  318.54811987
 318.15253822 317.7236438  317.35054361 317.0028849  316.62956542
 316.12719106 315.81591467 320.78156663 320.74993785 320.56363296
 320.29768314 320.14354585 319.98701972 320.0186487  320.01657244
 320.03050105 320.18080538 320.5270821  320.9342913  321.41605785
 321.94292322 322.21786222 322.40569687 322.60053058 322.80758885
 322.925929

[247, 249, 24, 250, 251, 252, 253, 254, 255, 25, 26, 27, 28, 29, 2, 30, 31, 32, 337, 338]
[247, 249, 24, 250, 251, 252, 253, 254, 255, 25, 26, 27, 28, 29, 2, 30]
[31, 32, 337, 338]
Splitted Successfully
(1098, 230401)
(1098,)
(278, 230401)
(278,)
The Model is trained well with the given images
The predicted Data is :
[202.6384962  202.24645078 201.7436279  201.35567152 200.81194549
 200.25932236 199.61582229 198.92417785 198.12560512 197.32278756
 196.53587929 195.64129009 194.66195085 193.66432884 192.86558153
 192.17259112 191.48312217 190.80799965 190.19023473 189.52301349
 188.7761032  188.05065337 187.38346656 186.76488915 186.01813475
 185.28093487 184.77651378 184.35325936 183.95106459 183.49581336
 183.03874865 182.42954326 182.09746041 181.8941303  181.67650699
 181.75765528 181.88492581 207.0584777  207.88450813 208.55623484
 208.97869405 209.18738    209.40989794 209.74213313 210.10814687
 210.51826591 211.06080913 211.63246007 212.08365821 212.54407224
 212.84324364 212.875

[363, 366, 367, 368, 369, 36, 37, 38, 39, 3, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
[363, 366, 367, 368, 369, 36, 37, 38, 39, 3, 40, 41, 42, 43, 44, 45]
[46, 47, 48, 49]
Splitted Successfully
(1085, 230401)
(1085,)
(278, 230401)
(278,)
The Model is trained well with the given images
The predicted Data is :
[229.97647783 229.81993042 229.57756752 229.48782827 229.46892332
 229.75323758 230.05894763 230.35120153 230.43651086 230.37493201
 230.31723522 230.28214491 230.1406546  229.79404466 229.65970312
 229.79350258 230.02534212 230.33453673 230.73880122 231.2394617
 231.66209318 232.03476168 232.19219239 232.26802071 232.30721395
 232.51365539 232.62261766 232.71185427 233.03976671 233.61798583
 234.23946652 234.88517326 235.55649478 236.12326367 236.69654613
 237.15748136 237.57713346 238.03891592 238.55574865 238.90390146
 239.19597342 239.28915972 239.18689373 238.97645256 238.62243524
 238.44630484 238.14312865 237.89551771 237.79122437 237.74171301
 237.7797642  237.67031616 237.5

[68, 69, 6, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 7, 80, 81, 82, 83, 85, 86]
[68, 69, 6, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 7, 80, 81]
[82, 83, 85, 86]
Splitted Successfully
(1068, 230401)
(1068,)
(204, 230401)
(204,)
The Model is trained well with the given images
The predicted Data is :
[329.96780805 330.58283602 331.65231834 332.61505071 333.95316016
 335.22627868 336.1437903  336.77351086 337.22322883 337.46287779
 338.23293851 338.92336894 339.86435892 340.75224261 340.98435737
 341.35526344 341.95897669 342.53503332 342.99153016 343.20084123
 343.50538333 343.79889098 344.03108149 344.13223508 344.25720996
 344.43532031 344.68671688 345.01355963 345.33355645 345.59618876
 345.79954036 345.96599411 345.95040459 345.23026609 324.05656266
 323.34167035 322.86124182 322.3982706  322.40151818 322.60665176
 323.26171237 323.94824817 324.69175016 325.2170361  325.77511796
 326.30507792 327.00595454 328.060896   328.72829958 329.58324706
 330.39466008 331.41572726 332.91051116 334